# Part 2: Creating a Basic Image Agent

![Alt text](img/augLLMs.png)

In this tutorial we'll be making a simplified image classifier/agent with Gemma3.

Theres two parts

* **Multimodal Gemma Classifier** - Using a Gemma model to 

* **Downstream Action** - A simple function that can process the results of the action

## Sending a Basic Prompt to the Model

Now that we have a tool ready, let's set up a simple function to interact with the model.

We'll define a basic `model_call(prompt)` function that sends user input to the LLM and receives a response.  
This will be the foundation we build on as we add more complex behaviors like tool calling.

In [5]:
from ollama import chat
from ollama import ChatResponse

# 12b is better so try and use it first
model = 'gemma3:4b'


# Note, the argument model_prompt is specific here
def model_call(model_prompt):
    
    response: ChatResponse = chat(model=model, messages=[
      {
        'role': 'user',
        'content': model_prompt,
      },
    ])
    return response['message']['content']

user_prompt = "Say hello to the class"

# Note, the argument user_prompt is specific here
model_call(user_prompt)

'Hello everyone! 😊 \n\nIt’s great to be here with you all today. \n\nHow’s everyone doing?'

## Adding an Image
Gemma3 has been trained with multimodality, where images are converted into embedding vectors the model can operate on. As a user you adding an image is quite straightforward.

In [23]:
image_path = "img/NotHotDog.jpg"  # Replace with the actual path to your image file

response = chat(
        model="gemma3:27b-it-qat",  # Use a vision-capable model like LLaVA
        messages=[
            {
                'role': 'user',
                'content': 'What is this?',
                'images': [image_path]
            }
        ]
    )

response["message"]["content"]

'This is a dachshund (a type of dog, often called a "wiener dog") wearing a hot dog costume! It’s a playful and funny outfit, capitalizing on the dog’s long shape to resemble a hot dog bun with a sausage and mustard on top.  It\'s a popular costume for Halloween or just for fun!'

## Turning our image checker into a classifier

In [11]:
image_path = "img/NotHotDog.jpg"  # Replace with the actual path to your image file

def image_chat(prompt, img_path):
    response = chat(
        model="gemma3:27b-it-qat",  # Use a vision-capable model like LLaVA
        messages=[
            {
                'role': 'user',
                'content': 'Is this an image of the food item hot dog say yes, otherwise say no, no other output',
                'images': [img_path]
            }
        ]
    )
    return response["message"]["content"]


image_chat(None,image_path) 

'no'

## Parsing the response
With our prompt complete we can turn this into a simple classifier.

In [13]:
image_path = "img/Hot_dog_with_mustard.png"  # Replace with the actual path to your image file

image_chat(None, image_path) 

'yes'

In [18]:
def parse_response(img_path):
    response = image_chat(None, image_path)
    if response.lower() == 'yes':
        return "Give treat"
    return "Add ketchup"

parse_response(image_path)

'Give treat'

## 🎯 Recap: What We Learned (THIS will be updated)

In this section, we built our first basic agent that can recognize when a tool call is needed and respond accordingly.

Here are the key ideas to remember:
- **Function calls aren't magic**: We rely on the model's "reasoning" to decide when to call a function, based on the system prompt and user input.
- **The model doesn't actually call functions itself**: It simply outputs a structured signal (like JSON) suggesting what should happen.  
- **The framework — your code — decides what to do**: It parses the model’s output, calls tools when needed, and can reinject the results for a better final answer.

This pattern — LLM suggests, framework acts — is the foundation for building more complex agents later.